# Ai 数字人

- https://learn.microsoft.com/zh-cn/azure/ai-services/speech-service/text-to-speech-avatar/what-is-text-to-speech-avatar
- https://github.com/Azure-Samples/cognitive-services-speech-sdk/tree/master/samples/python

- 可用区域：东南亚、北欧、西欧、瑞典中部、美国中南部、美国东部2 和美国西部2
- TTS支持的语音列表：https://learn.microsoft.com/zh-cn/azure/ai-services/speech-service/language-support?tabs=tts


In [2]:
import json
import logging
import os
import sys
import time
import uuid

from azure.identity import DefaultAzureCredential
import requests

In [3]:
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format="[%(asctime)s] %(message)s",
    datefmt="%m/%d/%Y %I:%M:%S %p %Z",
)
logger = logging.getLogger(__name__)

os.environ["SPEECH_KEY"] = "xxx"
os.environ["SPEECH_ENDPOINT"] = "https://westus2.api.cognitive.microsoft.com/"
# https://<region>.api.cognitive.microsoft.com or https://<custom_domain>.cognitiveservices.azure.com
SPEECH_ENDPOINT = os.environ["SPEECH_ENDPOINT"]
SPEECH_KEY = os.environ["SPEECH_KEY"]
API_VERSION = "2024-08-01"

In [6]:
def submit_synthesis(
    text: str,
    voice: str = "zh-CN-XiaoxiaoNeural",
    avatar_name: str = "lisa",
    avatar_style: str = "graceful-sitting",
):
    job_id = uuid.uuid4()
    url = f"{SPEECH_ENDPOINT}/avatar/batchsyntheses/{job_id}?api-version={API_VERSION}"
    header = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": SPEECH_KEY,
    }

    payload = {
        "synthesisConfig": {
            # 参考声音 https://learn.microsoft.com/zh-cn/azure/ai-services/speech-service/language-support?tabs=tts
            "voice": voice,
        },
        "customVoices": {
            # "CUSTOM_VOICE_NAME": "CUSTOM_VOICE_ID" # 如果自定义的话
        },
        "inputKind": "PlainText",  # PlainText or SSML
        "inputs": [{"content": text}],
        "avatarConfig": {
            "customized": False,  # set to True if use customized avatar
            "talkingAvatarCharacter": avatar_name,
            "talkingAvatarStyle": avatar_style,  # talking avatar style, required for prebuilt avatar, optional for custom avatar
            "videoFormat": "mp4",  # mp4 or webm, webm is required for transparent background
            "videoCodec": "h264",  # hevc, h264 or vp9, vp9 is required for transparent background; default is hevc
            "subtitleType": "soft_embedded",
            "backgroundColor": "#FFFFFFFF",  # background color in RGBA format, default is white; can be set to 'transparent' for transparent background
            # "backgroundImage": "https://samples-files.com/samples/Images/jpg/1920-1080-sample.jpg", # background image URL, only support https, either backgroundImage or backgroundColor can be set
        },
    }

    response = requests.put(url, json.dumps(payload), headers=header)
    if response.status_code < 400:
        logger.info("Batch avatar synthesis job submitted successfully")
        logger.info(f'Job ID: {response.json()["id"]}')
        return job_id
    else:
        logger.error(
            f"Failed to submit batch avatar synthesis job: [{response.status_code}], {response.text}"
        )


def get_synthesis(job_id):
    url = f"{SPEECH_ENDPOINT}/avatar/batchsyntheses/{job_id}?api-version={API_VERSION}"
    header = {"Ocp-Apim-Subscription-Key": SPEECH_KEY}

    response = requests.get(url, headers=header)
    if response.status_code < 400:
        logger.debug("Get batch synthesis job successfully")
        logger.debug(response.json())
        if response.json()["status"] == "Succeeded":
            logger.info(
                f'Batch synthesis job succeeded, download URL: {response.json()["outputs"]["result"]}'
            )
        return response.json()["status"]
    else:
        logger.error(f"Failed to get batch synthesis job: {response.text}")


def list_synthesis_jobs(skip: int = 0, max_page_size: int = 100):
    """List all batch synthesis jobs in the subscription"""
    url = f"{SPEECH_ENDPOINT}/avatar/batchsyntheses?api-version={API_VERSION}&skip={skip}&maxpagesize={max_page_size}"
    header = {"Ocp-Apim-Subscription-Key": SPEECH_KEY}

    response = requests.get(url, headers=header)
    if response.status_code < 400:
        logger.info(
            f"List batch synthesis jobs successfully, got {len(response.json())} jobs"
        )
        logger.info(response.json())
        return response.json()
    else:
        logger.error(f"Failed to list batch synthesis jobs: {response.text}")

In [ ]:
voice = "zh-CN-XiaoxiaoNeural"
avatar_name = "lisa"
avatar_style = "graceful-sitting"  # casual-sitting
text = "新岁序开，同赴新程。在这万象更新的时刻，愿你在新的一年里，目之所及皆美好，心之所向皆如愿。事业上，如鹏展翅，扶摇直上，每一份努力都能换来丰硕回报；生活中，有暖阳相伴，与欢笑同行，琐碎日常皆是温馨诗篇。家人安康喜乐，朋友常伴左右，爱情甜蜜顺遂。愿你在岁月的长河中，积攒满满的幸福，每一个日子都闪闪发光，新年快乐，万事胜意！"

job_id = submit_synthesis(
    text=text, voice=voice, avatar_name=avatar_name, avatar_style=avatar_style
)
if job_id is not None:
    while True:
        status = get_synthesis(job_id)
        if status == "Succeeded":
            logger.info("batch avatar synthesis job succeeded")
            break
        elif status == "Failed":
            logger.error("batch avatar synthesis job failed")
            break
        else:
            logger.info(
                f"batch avatar synthesis job is still running, status [{status}]"
            )
            time.sleep(5)

In [7]:
jobs = list_synthesis_jobs()
jobs

[01/21/2025 10:25:37 AM 中国标准时间] List batch synthesis jobs successfully, got 1 jobs
[01/21/2025 10:25:37 AM 中国标准时间] {'value': [{'id': 'f50cc3dc-85df-4315-82c5-9986553d3c11', 'status': 'Succeeded', 'createdDateTime': '2025-01-21T01:40:56.1095988Z', 'lastActionDateTime': '2025-01-21T01:41:34.2520832', 'inputKind': 'PlainText', 'customVoices': {}, 'properties': {'timeToLiveInHours': 744, 'sizeInBytes': 8860628, 'durationInMilliseconds': 35880, 'succeededCount': 1, 'failedCount': 0, 'billingDetails': {'talkingAvatarDurationSeconds': 35}}, 'synthesisConfig': {'voice': 'zh-CN-XiaoxiaoNeural'}, 'avatarConfig': {'talkingAvatarCharacter': 'lisa', 'talkingAvatarStyle': 'graceful-sitting', 'videoFormat': 'Mp4', 'videoCodec': 'h264', 'subtitleType': 'soft_embedded', 'backgroundColor': '#FFFFFFFF', 'bitrateKbps': 2000, 'customized': False}, 'outputs': {'result': 'https://stttssvcprodusw2.blob.core.windows.net/batchsynthesis-output/f109138e3ab3447884285125110715d6/f50cc3dc-85df-4315-82c5-9986553d3c11

{'value': [{'id': 'f50cc3dc-85df-4315-82c5-9986553d3c11',
   'status': 'Succeeded',
   'createdDateTime': '2025-01-21T01:40:56.1095988Z',
   'lastActionDateTime': '2025-01-21T01:41:34.2520832',
   'inputKind': 'PlainText',
   'customVoices': {},
   'properties': {'timeToLiveInHours': 744,
    'sizeInBytes': 8860628,
    'durationInMilliseconds': 35880,
    'succeededCount': 1,
    'failedCount': 0,
    'billingDetails': {'talkingAvatarDurationSeconds': 35}},
   'synthesisConfig': {'voice': 'zh-CN-XiaoxiaoNeural'},
   'avatarConfig': {'talkingAvatarCharacter': 'lisa',
    'talkingAvatarStyle': 'graceful-sitting',
    'videoFormat': 'Mp4',
    'videoCodec': 'h264',
    'subtitleType': 'soft_embedded',
    'backgroundColor': '#FFFFFFFF',
    'bitrateKbps': 2000,
    'customized': False},
   'outputs': {'result': 'https://stttssvcprodusw2.blob.core.windows.net/batchsynthesis-output/f109138e3ab3447884285125110715d6/f50cc3dc-85df-4315-82c5-9986553d3c11/0001.mp4?skoid=fdd04a02-98a3-46a7-830a-